In [1]:
using Distributed

In [2]:
include("./mps.jl");

In [3]:
include("./mpo.jl");

In [4]:
include("./dmrg.jl");

In [ ]:
include("./RUC.jl")

In [ ]:
include("./time_evo.jl")

# Test DMRG

In [5]:
cols=3
rows=3
config=zeros(Int,rows,cols)
k=0
for j in 1:cols
    for i in 1:rows
        k+=1
        config[i,j]=k
    end
end
config

3×3 Matrix{Int64}:
 1  4  7
 2  5  8
 3  6  9

In [6]:
d=2
chi=10
chimax=64
sweeps=10
J1=1.
J2=0.5
M=MPO()
Initialize!("J1-J2_ITensors",M,J1,J2,cols,rows,config);

In [7]:
ψ=MPS()
Initialize!(ψ,d,chi,cols*rows)
two_sites_dmrg!(ψ,M,sweeps,chimax)

Done! Energy= -15.837359989589917; Variance: -5.684341886080802e-14


# Test timevo

In [19]:
ψ=MPS()
N=10
d=1
Initialize!("product_state",ψ,repeat(["up","down"],Int(N/2)))

"Product state: [\"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\"]"

In [20]:
Ulist=RUC_layer(ψ.N);
print(size(Ulist));

(9,)

In [21]:
ψt=copy(ψ)
sweeps=1
trotter_evolution!(ψt,Ulist,sweeps)